# Creating a stellar color-magnitude (or Hertzsprung-Russell) diagram

The [Hertzsprung-Russell diagram](https://en.wikipedia.org/wiki/Hertzsprung–Russell_diagram) is a fundamental diagram in astronomy that displays important relationships between the stellar color (or temperature) and absolute brightness (or luminosity). 

In this exercise, we will use existing stellar catalogs to produce the H-R diagram. 


In [ ]:
# As a hint, we include the code block for Python modules that you will likely need to import:   
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

# For downloading files
from astropy.utils.data import download_file
from astropy.io import fits

import pyvo as vo
from pyvo import registry

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")

## Step 1: Find appropriate catalogs

We want to find a star catalog that has the available data to produce the H-R diagram, i.e., the absolute magnitudes (or both apparent magnitudes AND distances, so we can calculate the absolute magnitudes) in two optical bands (e.g., B and V). This would give us color. Or we need B- OR V- band magnitude and the stellar temperature. 

To simplify this problem, we want to find a catalog of an open cluster of stars, where all the stars were born around the same time and are located in one cluster. This simplifies the issue of getting accurate distances to the stars. One famous cluster is the Pleiades in the constellation of Taurus. So first we start by searching for an existing catalog with data on Pleiades that will provide the necessary information about the stars: magnitudes in two bands (e.g., B and V), which can be used to measure color, or temperature of the star plus one magnitude. 


### DATA DISCOVERY STEPS: 

Here is useful link for [how the pyvo registry search works](https://pyvo.readthedocs.io/en/latest/registry/index.html).

In [ ]:
resources = registry.search(keywords=['star pleiades'], includeaux=True)
print(len(resources.to_table()))

Note: The includeaux=True includes auxiliary services. 

This returns > 300 resources. However, we know that we want to eventually access the search using additional column information (i.e., beyond RA and Dec, which is all that Simple Cone Search returns). Therefore we need the service to be a TAP service, specifically. 

In [ ]:
tap_services = registry.search(servicetype='tap', keywords=['star pleiades'], includeaux=True)
print(len(tap_services.to_table()))
tap_services.to_table()

Note that this is now >100 services, but the data within the columns will be accessible for our purposes. 

#### Next, we need to find which of these has the columns of interest, i.e. magnitudes in two bands to create the color-magnitude diagram. 

For each resource element (i.e. row in the table above), there are useful attributes, which are [described here]( https://pyvo.readthedocs.io/en/latest/api/pyvo.registry.regtap.RegistryResource.html#pyvo.registry.regtap.RegistryResource):

Based on these, we can create a search based on keywords in the description. 

In [ ]:
def getrow( result, keywords=None ):
    ind = [] 
    keys = keywords.split()
    n_keys = len(keys)
    n_match = 0
    for i in range(len(result.to_table())):
        #print(resources[i].short_name)
        for each in keys: 
            if each in result[i].res_description:
                n_match = n_match + 1
        if n_match == n_keys:
            ind.append(i)
        n_match = 0
        
    return(ind)

In [ ]:
ind = getrow(tap_services, keywords="magnitude color")
print(len(ind))

So using this we can reduce the matched tables to ones that are a bit more catered to our experiment. Note, that there is redundancy in some resources since these are available via multiple services and/or publishers. Therefore a bit more cleaning can be done to provide only the unique matches. 

In [ ]:
tap_services.to_table()[ind]['ivoid']

In [ ]:
def getunique( result, ind ):
    short_name = []
    unique_ind = []
    for i in range(len(ind)):
        short = result[ind[i]].short_name       
        if short not in short_name: 
            short_name.append(short)
            unique_ind.append(ind[i])
    
    return(unique_ind)

In [ ]:
uniq_ind=getunique(tap_services, ind)
print(len(uniq_ind))

In [ ]:
tap_services.to_table()[uniq_ind]['ivoid']

This shows that in this case, all of our TAP results matching our keyword search are provided by Vizier and are unique. 

In [ ]:
# To read the descriptions of the resulting matches:

for i in uniq_ind: 
    print("  ***  \n")
    print(tap_services[i].creators)
    print(tap_services[i].res_description)
    
    

<i> RESULT: Based on these, the first one (by Eichhorn et al) looks like a good start. </i>

### At this point, you can proceed to Step 2. 

-- OR --

### Try a different data discovery method! 

### Alternative Method: Use ADS to search for appropriate paper and access data via NED.

There are multiple paths for the data discovery. So it may also be that you know the paper that has the data you are interested in and want to access via the bibcode or authors, etc. 

In this case, let's assume that we have the information that the Eichhorn+1970 paper has the data that we need to create the H-R diagram: https://ui.adsabs.harvard.edu/abs/1970MmRAS..73..125E/abstract

We can either search by bibcode (1970MmRAS..73..125E) or "Eichhorn" to get the access_urls that will allow us to work with the data. 

Before this step, if may help to see the names of the fields available to use. Notice the following fields: 

"source_value" contains the bibcode information that we want; "creator_seq" lists the authors; 

and 

"access_url" provides the url from where the data can be accessed. 


In [ ]:
## You already have this from above:
#tap_services = registry.search(servicetype='tap', keywords=['star pleiades'], includeaux=True)
print(tap_services.fieldnames)

First, Try using bibcode: 

In [ ]:
bibcode = '1970MmRAS..73..125E' # Eichhorn
idx=-1
for s in tap_services:
    idx+=1
    if bibcode in s['source_value']:
        myidx=idx
        print(f"{s.short_name}, {s.source_value}, {s.access_url}")
        break 
#  Note that using the to_table() lets you search the result 
#   easily using all columns.  But in the end, you want to get
#   back not an astropy table row, which you cannot use, but the
#   original RegistryResult that has the callable TAP service.  
myTAP=tap_services[myidx]

Note that the URL is a generic TAP url for Vizier.  All of its tables can be accessed by that same TAP services.  It'll be in the ADQL query itself that you specify the table name.  We'll see this below.

Next, try using Author name: 

In [ ]:
author = 'Eichhorn'

for record in resources: 
    names=record.creators
    if 'Eichhorn' in names[0]: 
        print("For %s: " %record.short_name)
        print("     Reference URL: %s" %record.reference_url)
        print("     Access URL: %s" %record['access_urls'][0])



In the code above, the record is a Registry Resource. You can access the attribute, "creators", from the resource, which is relevant for our example here since this is a direct way to get the author names. The other attributes, "access_url" and "reference_url", provides two types of URLs. The former can be used to access the service resource (as described above) and the latter points to a human-readable document describing this resource. 

If you click on the Reference URL links, you can find the abstract, Readme file, Vizier table, and much more information associated with this catalog.

Additional information about PyVO Registry Resource and its attributes is available <a href="https://pyvo.readthedocs.io/en/latest/api/pyvo.registry.regtap.RegistryResource.html#pyvo.registry.regtap.RegistryResource "> from this page</a>. 


<P> *** <P>

These examples provide a few ways to access the information of interest. 

Below are a few other ways to see what the tap_service table contains. 

1. To view the column information: tap_services.to_table().columns() shows the metadata contained in the tap service. We will reference some of this columns below as we try to find the appropriate table. 

2. tap_services[index].describe(): The table with the tap_services output has, in our case, 72 tables listed and each includes metadata containing some human readable description. You can get the description for one case or for all the records by iterating through the resource. In the former case, we show the description for the Eichhorn data, whose index is uniq_ind[0]. The latter case also follows.  
    

In [ ]:
print( tap_services.to_table().columns )

In [ ]:
tap_services[uniq_ind[0]].describe()   # For Eichhorm+1970 example. 

In [ ]:
# To iterate over all the tables: 
for tapsvc in tap_services: 
    tapsvc.describe()

## Step 2: Acquire the relevant data and make a plot!

In [ ]:
tables = tap_services[uniq_ind[0]].service.tables

short_name = "I/90"
# find table name: 
for name in tables.keys():
    if short_name in name:  
        print(name)

We can write code to eliminate the other cases (e.g., VI or VIII...) but we wanted to keep this cell to illustrate that the table name (which is required for the query) will likely include the short_name appended to "/catalog" (or "/table"). 

But the other roman numeral catalogs are obviously different catalogs. Therefore try the below for a better match: 

In [ ]:
# find (more restricted) table name: 
for name in tables.keys():
    if name.startswith(short_name):  
        print(name)
        tablename=name

In [ ]:
query = 'SELECT * FROM "%s"' %tablename
print(query)
results = tap_services[uniq_ind[0]].search(query)
#results = taps[0].search(query)
results.to_table()

We can access the column data as array using the .getcolumn(colname) attribute, where the colname is given in the table above. In particular the "CI" is the color index and "Ptm" is the photovisual magnitude. See [here](https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=I/90) for details about the columns. 

In [ ]:
color = results.getcolumn('CI')
mag = results.getcolumn('Ptm') 

### Plotting... 
Note: The magnitudes here are apparent and therefore in plotting, the color-magnitude diagram is typically brightness increasing upwards (higher y-axis) so we will flip the y-axis here. 

In [ ]:
plt.ylim(15, 0)
plt.ylabel("V [apparent mag]")
plt.xlabel("B-V")

plt.plot(color, mag, 'o', color='black')

## Step 3. Compare with other color-magnitude diagrams for Pleiades:

There is nice discussion here: http://www.southastrodel.com/Page03009a.htm  about the color-magnitude diagram. Their Fig 4 looks slightly cleaner because part of this investigation was to select the 270 stars that are vetted members and restricted to stellar types more massive than K0. 

The dataset is from Raboud+1998 (1998A&A...329..101R)

Therefore in this next step, we will use the bibcode to select this data and overplot with the previous data to compare. 

We note that we did not discover this table above because alas! the description does not include the keywords (e.g. color, magnitude) that we used to refine our search. However, this table does have color and magnitude columns. 

In [ ]:
bibcode = '1998A&A...329..101R' # Raboud
all_bibcodes = tap_services.getcolumn('source_value')
all_shortnames = tap_services.getcolumn('short_name')

match = np.where(all_bibcodes == bibcode)

# Show relevant short_name (for Raboud paper): 
short_name = all_shortnames[match][0]
print(short_name)
print("----------")
ind = int(match[0])

tap_services[ind].describe()


In [ ]:
# Doing steps above to view table from Raboud+1998

tables = tap_services[ind].service.tables 

# find table name: 
for name in tables.keys():
    if short_name in name:  
        tablename = name
        
query = 'SELECT * FROM "%s"' %tablename
print(query)
results = tap_services[uniq_ind[0]].search(query)
results.to_table()

In [ ]:
R98_color = results.getcolumn('B-V')
R98_mag = results.getcolumn('Vmag') 

plt.ylim(15, 0)
plt.ylabel("V [apparent mag]")
plt.xlabel("B-V")
plt.plot(color, mag, 'o', markersize=4.0, color='black') ## This is Eichhorn data
plt.plot(R98_color, R98_mag, 's', markersize=5.0, color='red') ## This is new data from Raboud+98


## BONUS: Step 4: The CMD as a distance indicator! 

Since the y-axis above is apparent magnitude, we can use the obvious features (e.g., main sequence curve) to translate the apparent magnitudes to absolute magnitudes (by comparing to published H-R diagrams given in absolute magnitudes) and measure the distance to Pleiades! 


In [ ]:
R98_color = results.getcolumn('B-V')
R98_mag = results.getcolumn('Vmag') 

sun_color = 0.65  # from http://www.astro.ucla.edu/~wright/magcolor.htm
sun_mag = 10.4   # Played with this value until it looked centered in relation at the B-V color above (yellow star!) 

plt.ylim(15, 0)
plt.ylabel("V [apparent mag]")
plt.xlabel("B-V")
plt.plot(color, mag, 'o', markersize=4.0, color='black') ## This is Eichhorn data
plt.plot(R98_color, R98_mag, 's', markersize=5.0, color='red') ## This is new data from Raboud+98
plt.plot(sun_color, sun_mag, '*', markersize=15.0, color='yellow') ## This is our estimated center point


In [ ]:
# Another measure... use the Sun: 
Vabs = 4.8   ## Sun @ B-V = 0.65 (taken from Wikipedia)
Vapp = 10.4  ## Based on rough reading of plot above at B-V = 0.65

dm= Vapp - Vabs   # distance module = 5log d / 10pc. 
dist = 10. ** (dm / 5. + 1.)
print("%10.1f pc " %dist)

True distance to Pleaides is 136.2 pc ( https://en.wikipedia.org/wiki/Pleiades ). Not bad! 